In [ ]:
import numpy as np
from matplotlib import pyplot
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fftshift
import os 
import seaborn as sns
from matplotlib.pyplot import figure

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/')

In [ ]:
data = pd.read_csv('distances_meal.csv')  


In [ ]:
# we get how many nans there are for each video in order to calculate the frame lenghts
nans = []
for video in data:
    nans.append(data[video].isna().sum())

In [ ]:
frame_lenghts =[]
for x,y in zip(data,nans):
    frame_lenghts.append((len(data[x])-y))

In [ ]:
rad = [1047, 1059, 1069, 1079, 1089, 1093, 1099, 1107, 1108, 
      1117, 1122, 1124, 1125, 1129, 1131, 1132, 1134, 1148, 1186, 
      1190, 1195, 1206, 1210, 1217, 1230, 1246, 1250, 1264, 2009, 2027]
no_rad = [1043, 1049, 1053, 1062, 1073, 1080, 1082, 1091, 1092, 
         1096, 1097, 1098, 1104, 1105, 1112, 1118, 1130, 1146, 
         1161, 1170, 1180, 1181, 1184, 1188, 1234, 1241, 1245, 1263,
         1269, 1282, 2025]
data_rad = data[data.columns & [str(x) for x in rad]]
data_rad
data_no_rad = data[data.columns & [str(x) for x in no_rad]]
data_no_rad

In [ ]:
# we get the labels list now
df =  pd.concat([data_rad.T.assign(dataset='rad'), data_no_rad.T.assign(dataset='no-rad')])
label = df['dataset'].sort_index()
label

In [ ]:
# add the label to the dataframe in the wide form
data = data.T
data['label'] = label
data

In [ ]:
# how many big values (i.e. over 0.31 there are for each video)
big = data.T.iloc[:-1,:].ge(0.08712949, axis=0).sum()
small = data.T.iloc[:-1,:].le(0.17380587, axis=0).sum()

In [ ]:
data['frequency big values'] = big
data['frequency small values'] = small

In [ ]:
data = data.T
data['frame_lengths'] = frame_lenghts
data

In [ ]:
data['proportion_big_quantile'] = data['frequency big values'] / data['frame_lengths']
data['proportion_small_quantile'] = data['frequency small values'] / data['frame_lengths']


In [ ]:
rad = data.query('label == "rad"')['proportion_small_quantile']

In [ ]:
no_rad = data.query('label == "no-rad"')['proportion_small_quantile']

In [ ]:
data.groupby('label').describe()

In [ ]:
from scipy import stats
res = stats.ttest_ind(rad, no_rad, 
                      equal_var=False)

In [ ]:
res

In [ ]:
data['mean'] = data.iloc[:,:-1].mean(axis=1)
data

In [ ]:
data.groupby('label')

## no difference in means 

In [ ]:
data.iloc[:,:-2]

In [ ]:
os.chdir('/Users/andrei-macpro/Documents/Data/Results/corpus_processed')

In [ ]:
scores = pd.read_excel('corpus_labelled.xlsx')  

In [ ]:
data['rinab'] = scores["Rinab"].values
data['dai'] = scores['DAI'].values

In [ ]:
from scipy.stats import spearmanr
corr, _ = spearmanr(data['rinab'], data['proportion_big_quantile'], nan_policy='omit')
print('Spearmans correlation: %.3f' % corr)

In [ ]:
plt.scatter(data['rinab'], data['proportion_small_quantile'])

In [ ]:
data.plot.bar(y='proportion_big_quantile', rot=0)


In [ ]:
rad.mean(), no_rad.mean()

In [ ]:
tested with 95% and 99%